# Multi-Agent Collaboration
Create multiple specialized agents that work together

In [ ]:
import boto3
import json

bedrock_agent = boto3.client('bedrock-agent', region_name='us-east-1')
bedrock_runtime = boto3.client('bedrock-agent-runtime', region_name='us-east-1')
lambda_client = boto3.client('lambda', region_name='us-east-1')
iam = boto3.client('iam')

ACCOUNT_ID = boto3.client('sts').get_caller_identity()['Account']

## 1. Create Specialized Agents

We'll create:
- **Research Agent**: Gathers information
- **Analysis Agent**: Analyzes data
- **Coordinator Agent**: Orchestrates the workflow

In [ ]:
def create_agent_with_role(name, instruction, model_id='us.anthropic.claude-3-5-sonnet-20241022-v2:0'):
    # Create role
    role_name = f"{name}-role"
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [{"Effect": "Allow", "Principal": {"Service": "bedrock.amazonaws.com"}, "Action": "sts:AssumeRole"}]
    }
    
    try:
        response = iam.create_role(RoleName=role_name, AssumeRolePolicyDocument=json.dumps(trust_policy))
        role_arn = response['Role']['Arn']
        iam.attach_role_policy(RoleName=role_name, PolicyArn='arn:aws:iam::aws:policy/AmazonBedrockFullAccess')
        import time; time.sleep(10)
    except iam.exceptions.EntityAlreadyExistsException:
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    
    # Create agent
    try:
        response = bedrock_agent.create_agent(
            agentName=name,
            agentResourceRoleArn=role_arn,
            foundationModel=model_id,
            instruction=instruction
        )
        agent_id = response['agent']['agentId']
    except bedrock_agent.exceptions.ConflictException:
        agents = bedrock_agent.list_agents()['agentSummaries']
        agent_id = next(a['agentId'] for a in agents if a['agentName'] == name)
    
    print(f"✓ Agent {name}: {agent_id}")
    return agent_id

In [ ]:
RESEARCH_AGENT_ID = create_agent_with_role(
    'research-agent',
    'You are a research specialist. Gather and summarize information on given topics.'
)

ANALYSIS_AGENT_ID = create_agent_with_role(
    'analysis-agent',
    'You are an analysis specialist. Analyze data and provide insights.'
)

COORDINATOR_AGENT_ID = create_agent_with_role(
    'coordinator-agent',
    'You coordinate between research and analysis agents. Break down tasks and synthesize results.'
)

## 2. Create Lambda for Agent Communication

In [ ]:
# Lambda role
lambda_role_name = 'multi-agent-lambda-role'
lambda_trust = {
    "Version": "2012-10-17",
    "Statement": [{"Effect": "Allow", "Principal": {"Service": "lambda.amazonaws.com"}, "Action": "sts:AssumeRole"}]
}

try:
    response = iam.create_role(RoleName=lambda_role_name, AssumeRolePolicyDocument=json.dumps(lambda_trust))
    LAMBDA_ROLE_ARN = response['Role']['Arn']
    iam.attach_role_policy(RoleName=lambda_role_name, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')
    iam.attach_role_policy(RoleName=lambda_role_name, PolicyArn='arn:aws:iam::aws:policy/AmazonBedrockFullAccess')
    import time; time.sleep(10)
except iam.exceptions.EntityAlreadyExistsException:
    LAMBDA_ROLE_ARN = iam.get_role(RoleName=lambda_role_name)['Role']['Arn']

In [ ]:
# Lambda to invoke other agents
agent_comm_code = f'''
import json
import boto3

bedrock_runtime = boto3.client('bedrock-agent-runtime', region_name='us-east-1')

RESEARCH_AGENT_ID = '{RESEARCH_AGENT_ID}'
ANALYSIS_AGENT_ID = '{ANALYSIS_AGENT_ID}'

def invoke_agent(agent_id, prompt):
    response = bedrock_runtime.invoke_agent(
        agentId=agent_id,
        agentAliasId='TSTALIASID',
        sessionId='multi-agent-session',
        inputText=prompt
    )
    
    result = ""
    for event in response['completion']:
        if 'chunk' in event:
            result += event['chunk']['bytes'].decode('utf-8')
    return result

def lambda_handler(event, context):
    function = event['function']
    parameters = {{p['name']: p['value'] for p in event.get('parameters', [])}}
    
    if function == 'research':
        topic = parameters.get('topic')
        result = invoke_agent(RESEARCH_AGENT_ID, f"Research: {{topic}}")
    elif function == 'analyze':
        data = parameters.get('data')
        result = invoke_agent(ANALYSIS_AGENT_ID, f"Analyze: {{data}}")
    else:
        result = "Unknown function"
    
    return {{
        'messageVersion': '1.0',
        'response': {{
            'actionGroup': event['actionGroup'],
            'function': function,
            'functionResponse': {{
                'responseBody': {{'TEXT': {{'body': result}}}}
            }}
        }}
    }}
'''.encode()

function_name = 'multi-agent-communication'

try:
    response = lambda_client.create_function(
        FunctionName=function_name,
        Runtime='python3.12',
        Role=LAMBDA_ROLE_ARN,
        Handler='index.lambda_handler',
        Code={'ZipFile': agent_comm_code},
        Timeout=300
    )
    LAMBDA_ARN = response['FunctionArn']
except lambda_client.exceptions.ResourceConflictException:
    lambda_client.update_function_code(FunctionName=function_name, ZipFile=agent_comm_code)
    LAMBDA_ARN = lambda_client.get_function(FunctionName=function_name)['Configuration']['FunctionArn']

print(f"✓ Lambda: {LAMBDA_ARN}")

## 3. Add Action Group to Coordinator

In [ ]:
api_schema = {
    "openapi": "3.0.0",
    "info": {"title": "Agent Communication API", "version": "1.0.0"},
    "paths": {
        "/research": {
            "post": {
                "summary": "Delegate research task",
                "operationId": "research",
                "parameters": [{"name": "topic", "in": "query", "required": True, "schema": {"type": "string"}}],
                "responses": {"200": {"description": "Success"}}
            }
        },
        "/analyze": {
            "post": {
                "summary": "Delegate analysis task",
                "operationId": "analyze",
                "parameters": [{"name": "data", "in": "query", "required": True, "schema": {"type": "string"}}],
                "responses": {"200": {"description": "Success"}}
            }
        }
    }
}

# Add permission
try:
    lambda_client.add_permission(
        FunctionName=function_name,
        StatementId='coordinator-invoke',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:us-east-1:{ACCOUNT_ID}:agent/{COORDINATOR_AGENT_ID}"
    )
except: pass

# Create action group
response = bedrock_agent.create_agent_action_group(
    agentId=COORDINATOR_AGENT_ID,
    agentVersion='DRAFT',
    actionGroupName='agent-delegation',
    actionGroupExecutor={'lambda': LAMBDA_ARN},
    apiSchema={'payload': json.dumps(api_schema)}
)

print("✓ Action group created")

## 4. Prepare All Agents

In [ ]:
import time

for agent_id in [RESEARCH_AGENT_ID, ANALYSIS_AGENT_ID, COORDINATOR_AGENT_ID]:
    bedrock_agent.prepare_agent(agentId=agent_id)
    
    while True:
        status = bedrock_agent.get_agent(agentId=agent_id)['agent']['agentStatus']
        if status == 'PREPARED':
            break
        time.sleep(5)
    
    # Create alias
    try:
        bedrock_agent.create_agent_alias(agentId=agent_id, agentAliasName='prod')
    except: pass
    
    print(f"✓ Agent {agent_id} ready")

## 5. Test Multi-Agent System

In [ ]:
def invoke_coordinator(prompt):
    aliases = bedrock_agent.list_agent_aliases(agentId=COORDINATOR_AGENT_ID)['agentAliasSummaries']
    alias_id = next(a['agentAliasId'] for a in aliases if a['agentAliasName'] == 'prod')
    
    response = bedrock_runtime.invoke_agent(
        agentId=COORDINATOR_AGENT_ID,
        agentAliasId=alias_id,
        sessionId='coordinator-session',
        inputText=prompt
    )
    
    result = ""
    for event in response['completion']:
        if 'chunk' in event:
            result += event['chunk']['bytes'].decode('utf-8')
    return result

# Test complex task requiring multiple agents
print(invoke_coordinator("Research cloud computing trends and analyze the market opportunities"))